In [122]:
# disable GPU. Remove this if you've compiled HOOMD for GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import hoomd
from hoomd import hpmc
import numpy as np
import hoomd.htf as htf
import tensorflow as tf

In [123]:
phi = 0.75 # packing fraction
sigma = 1
N = 64
L = (N*(np.pi*0.25*sigma**2)/phi)**0.5

In [124]:
class WCAPotential(htf.SimModel):
    def setup(self):
        self.avg_rdf = tf.keras.metrics.MeanTensor()
    def compute(self, nlist):
        # Use Weeks-Chandler-Anderson (WCA) repulisve potential
        r12 = htf.nlist_rinv(nlist)**12 # nlist_rinv is neighbor 1 / r^12
        # make it so anything above 2^1/6 is 0
        r = tf.norm(nlist[:,:,:2], axis=2)
        pair_energy = tf.cast(r < 2**(1/6), tf.float32) * r12
        particle_energy = tf.reduce_sum(pair_energy, axis=1) # sum over neighbors  
        all_energy = tf.reduce_sum(particle_energy)
#         print(particle_energy)
#         self.particle_e = particle_energy
#         forces = htf.compute_nlist_forces(nlist, particle_energy)
        # compute rdf
#         inst_rdf = htf.compute_rdf(nlist, [0, 3.5])
#         self.avg_rdf.update_state(inst_rdf)
        return all_energy

In [125]:
hoomd.context.initialize('')
box = hoomd.data.boxdim(L=L,dimensions=2)
snap = hoomd.data.make_snapshot(N=N,particle_types=['A'],box=box)
a = L/np.sqrt(N)
snap.particles.position[:] = [(i,j,0) for i in np.arange(-L/2,L/2,a) for j in np.arange(-L/2,L/2,a)]

In [126]:
system = hoomd.init.read_snapshot(snap)
hoomd.md.integrate.mode_standard(dt=0.001)
hoomd.md.integrate.nvt(group=hoomd.group.all(), kT=1e-6, tau=0.5).randomize_velocities(seed=1)


notice(2): Group "all" created containing 64 particles


In [127]:
model = WCAPotential(32, output_forces=False)
nlist = hoomd.md.nlist.cell()
tfcompute = htf.tfcompute(model)
tfcompute.attach(nlist, r_cut=3, save_output_period=1)


notice(2): Force mode is FORCE_MODE.hoomd2tf 
notice(2): Starting TensorflowCompute 
notice(2): completed reallocate


In [128]:
hoomd.run(1, quiet=True)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 64
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no


In [129]:
model.compute(nlist=tfcompute.get_nlist_array().astype(np.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=194.11232>

In [132]:
for i in range(100):
    hoomd.run(1, quiet=True)
    print(model.compute(nlist=tfcompute.get_nlist_array().astype(np.float32)))
    for particle in system.particles:
        particle.position += np.random.random(3) * np.array([1,1,0]) * 0.05

tf.Tensor(226.8374, shape=(), dtype=float32)
tf.Tensor(232.4011, shape=(), dtype=float32)
tf.Tensor(236.71341, shape=(), dtype=float32)
tf.Tensor(241.44055, shape=(), dtype=float32)
tf.Tensor(250.7398, shape=(), dtype=float32)
tf.Tensor(262.47394, shape=(), dtype=float32)
tf.Tensor(274.72894, shape=(), dtype=float32)
tf.Tensor(286.48785, shape=(), dtype=float32)
tf.Tensor(310.7634, shape=(), dtype=float32)
tf.Tensor(294.2164, shape=(), dtype=float32)
tf.Tensor(305.02173, shape=(), dtype=float32)
tf.Tensor(318.97366, shape=(), dtype=float32)
tf.Tensor(334.28128, shape=(), dtype=float32)
tf.Tensor(332.7252, shape=(), dtype=float32)
tf.Tensor(363.107, shape=(), dtype=float32)
tf.Tensor(360.46747, shape=(), dtype=float32)
tf.Tensor(383.5398, shape=(), dtype=float32)
tf.Tensor(399.557, shape=(), dtype=float32)
tf.Tensor(416.23804, shape=(), dtype=float32)
tf.Tensor(402.40552, shape=(), dtype=float32)
tf.Tensor(417.67685, shape=(), dtype=float32)
tf.Tensor(424.05737, shape=(), dtype=float32)

In [108]:
hoomd.run(1, quiet=True)

In [109]:
model.compute(nlist=tfcompute.get_nlist_array().astype(np.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=2856122.5>

In [101]:
np.random.random(3) * np.array([1,1,0])

array([0.73006422, 0.52548487, 0.        ])

In [100]:
for i in system.particles:
    print(i.position)

(-3.5967348725656487, -3.24707630315433, -0.4493110846659958)
(-4.042360730082015, -2.90682219606333, 0.12716922378197393)
(-3.7365407798278802, -1.997068670846132, -0.107777505751808)
(-3.1537886452654744, -0.9938706680494237, 0.08760893559286054)
(-3.9279176023429017, 0.4587927575875879, -0.013451928946464187)
(-3.1035622082115792, 1.8716796148166566, -0.28022139041824345)
(-3.1769141349122663, 2.1290269361771066, -0.11355359032426393)
(-3.1007543606852304, 3.4897369155052753, -0.19316347232043463)
(-2.69404156681761, -4.047146952712823, 0.108963950401471)
(-2.475723371682935, -2.40039099117915, 0.24173231957715624)
(-2.7027651207651644, -1.222749902873934, 0.10275107687200413)
(-2.591538169127131, -0.24648686535559894, -0.3481529186701725)
(-2.7268872238665165, 0.3036626391703729, 0.48773757034832244)
(-2.8748161384988435, 1.062384105864309, 0.38409050320088456)
(-2.1911224602577706, 2.2104861507258335, -0.26681935676729407)
(-2.2372433149893984, 3.2573332824696664, 0.45407038007289

In [50]:
tfcompute.get_nlist_array()

array([[[ 1.02332687,  1.02332687,  0.        ,  0.        ],
        [ 1.02332687,  2.04665351,  0.        ,  0.        ],
        [ 2.04665351,  1.02332687,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 1.02332687,  0.        ,  0.        ,  0.        ],
        [ 1.02332687,  1.02332664,  0.        ,  0.        ],
        [ 1.02332687,  2.04665339,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 1.02332687, -1.02332664,  0.        ,  0.        ],
        [ 1.02332687,  0.        ,  0.        ,  0.        ],
        [ 1.02332687,  1.02332675,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.  

In [62]:
hoomd.run(1)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 64
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:08 | Step 1 / 1 | TPS 14.9403 | ETA 00:00:00
Average TPS: 14.8243
---------
-- Neighborlist stats:
0 normal updates / 1 forced updates / 0 dangerous updates
n_neigh_min: 36 / n_neigh_max: 36 / n_neigh_avg: 36
shortest rebuild period: 100
-- Cell list stats:
Dimension: 2, 2, 1
n_min    : 12 / n_max: 21 / n_avg: 16
** run complete **


In [63]:
tfcompute.get_nlist_array()

array([[[ 1.02173102,  1.03329851,  0.        ,  0.        ],
        [ 1.01350555,  2.0431701 ,  0.        ,  0.        ],
        [ 2.04569406,  0.00358522,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 1.02415544,  0.00624493,  0.        ,  0.        ],
        [ 1.01592997,  1.01611653,  0.        ,  0.        ],
        [ 1.02538225,  2.04565783,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.00247249,  2.04416319,  0.        ,  0.        ],
        [ 1.02305749, -1.0147478 ,  0.        ,  0.        ],
        [ 1.01483202, -0.0048762 ,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.  

In [64]:
tfcompute.outputs

[array([194.46597], dtype=float32)]